# 4 - Standards
***
Matthew Brown

2023.06.09
***

**Requirements:**
- Local weather data (provided)

**Objectives:**
1. Import weather data
2. Calculate installation standoff - level 1
3. Calculate installation standoff - level 2

**Background:**

This example demonstrates the calculation of a minimum standoff distance necessary for roof-mounted PV modules to ensure that the $T_{98}$ operational temperature remains under 70°C, in which case the more rigorous thermal stability testing requirements of IEC TS 63126 would not needed to be considered. We use data from [Fuentes, 1987] to model the approximate exponential decay in temperature, $T(X)$, with increasing standoff distance, $X$, as,

$$ X = -X_0 \ln\left(1-\frac{T_0-T}{\Delta T}\right)$$

where $T_0$ is the temperature for $X=0$ (insulated back) and $\Delta T$ is the temperature difference between an insulated back ($X=0$) and open rack mounting configuration ($X=\infty)$.

The following figure showcases this calulation for the entire United States. We used pvlib and data from the National Solar Radiation Database (NSRDB) to calculate the module temperatures for different mounting configuration and applied our model to obtain the standoff distance for roof-mounted PV systems.


In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.1.1

In [1]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR

## 1. Import Weather Data

The function has two minimum requirements:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, Wind-Speed
- Site meta-data containing (at least) Latitude, Longitude, Time Zone


In [2]:
weather_file = os.path.join(DATA_DIR,'psm3_demo.csv')

WEATHER, META = pvdeg.weather.read(weather_file,'psm')

## 2. Calculate Installation Standoff - Level 1

Level - 1 : Temeprature Dependence as per IEC TS 63216

The following is the minimum function call. See step **3** or the documentation for further details and optional parameters.

In [3]:
standoff = pvdeg.standards.calc_standoff(weather_df=WEATHER, meta=META)

## 3. Calculate Installation Standoff - Level 2

Let's take a closer look at the function and some optional parameters.

- level : 1 or 2 (see IEC TS 63216)
- tilt : tilt from horizontal of PV module
- azimuth : azimuth in degrees from North
- sky_model : pvlib compatible model for generating sky characteristics (Options: 'isotropic', 'klucher', 'haydavies', 'reindl', 'king', 'perez')
- temp_model : pvlib compatible module temperature model (Options: 'sapm', 'pvsyst', 'faiman', 'sandia')
- module_type : basic module construction (Options: 'glass_polymer', 'glass_glass')
- x_0 : thermal decay constant [cm] (see documentation)
- wind_speed_factor : Wind speed correction factor to account for different wind speed measurement heights between weather database (e.g. NSRDB) and the tempeature model (e.g. SAPM)

In [4]:
standoff = pvdeg.standards.calc_standoff(weather_df=WEATHER, meta=META,
                                         level=2,
                                         tilt=None,
                                         azimuth=180,
                                         sky_model='isotropic',
                                         temp_model='sapm',
                                         module_type='glass_polymer',
                                         x_0=6.1,
                                         wind_speed_factor=1)